In [ ]:
# Install via pip
%pip install h3
%pip install folium

In [ ]:
# Verify installation
import h3
print(h3.__version__)  # Output: 4.1.0 (or latest version)

In [ ]:
import folium
import h3

base_cell = '8001fffffffffff'  # Resolution 0 pentagon
children = h3.cell_to_children(base_cell, res=1)

# Create a map centered at the center of the base hexagon
base_center = h3.cell_to_latlng(base_cell)
GeoSpatialMap = folium.Map(location=[base_center[0], base_center[1]], zoom_start=9)

# Function to get hexagon boundaries
def get_hexagon_bounds(h3_address):
    boundaries = h3.cell_to_boundary(h3_address)
    # Folium expects coordinates in [lat, lon] format
    return [[lat, lng] for lat, lng in boundaries]

# Add base hexagon
folium.Polygon(
    locations=get_hexagon_bounds(base_cell),
    color='red',
    fill=True,
    weight=2,
    popup=f'Base: {base_cell}'
).add_to(GeoSpatialMap)

# Add children hexagons
for child in children:
    folium.Polygon(
        locations=get_hexagon_bounds(child),
        color='blue',
        fill=True,
        weight=1,
        popup=f'Child: {child}'
    ).add_to(GeoSpatialMap)

GeoSpatialMap

In [ ]:
delhi_cell = h3.latlng_to_cell(28.6139, 77.2090, 9)  # New Delhi coordinates

# Traverse hierarchy upwards
parent = h3.cell_to_parent(delhi_cell, res=8)
print(f"Parent at res 8: {parent}")

# Traverse hierarchy downwards
children = h3.cell_to_children(parent, res=9)
print(f"Contains {len(children)} children")

# Create a new map centered on New Delhi
delhi_map = folium.Map(location=[28.6139, 77.2090], zoom_start=15)

# Add the parent hexagon (resolution 8)
folium.Polygon(
    locations=get_hexagon_bounds(parent),
    color='red',
    fill=True,
    weight=2,
    popup=f'Parent: {parent}'
).add_to(delhi_map)

# Add all children hexagons (resolution 9)
for child_cell in children:
    color = 'yellow' if child_cell == delhi_cell else 'blue'
    folium.Polygon(
        locations=get_hexagon_bounds(child_cell),
        color=color,
        fill=True,
        weight=1,
        popup=f'Child: {child_cell}'
    ).add_to(delhi_map)

delhi_map